# Laboratory work #3 (text vectorization)

In [64]:
import re
import os
from collections import defaultdict, Counter
import string

from pathlib import Path
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


import matplotlib.pyplot as plt
from pandas.errors import EmptyDataError
import numpy as np
from math import log1p
import gensim

[nltk_data] Downloading package punkt to /Users/aleksei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def read_files(root_dir, n=None):
    file_paths = []
    
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.tsv'):
                file_paths.append(os.path.join(subdir, file))

    data = []
    for i, file_path in enumerate(file_paths):
        if n is not None and i >= n:
            break
        try:
            d = pd.read_csv(file_path, sep='\t', header=0)
        except EmptyDataError as e:
            print(i, file_path, e)
        data.append(d.dropna())
    return data

In [26]:
train = read_files('../assets/annotated-corpus/train', 
                   1000
                   )
# val = read_files('../assets/annotated-corpus/val', 
#                 #  100
#                  )
# test = read_files('../assets/annotated-corpus/test', 
#                 #   100
#                   )

In [27]:
train[0].head()

,Token,Stem,Lemma
0,WASHINGTON,washington,WASHINGTON
1,Reuters,reuter,Reuters
2,The,the,The
3,head,head,head
4,of,of,of


In [29]:
def is_valid_token(token, token_frequencies, min_frequency=2):
    if token in string.punctuation:
        return False
    if token in stop_words:
        return False
    if token_frequencies[token] < min_frequency:
        return False
    return True

def get_freqs(dfs):
    token_frequencies = Counter()
    term_document_matrix = defaultdict(lambda: defaultdict(int))

    for doc_id, df in enumerate(dfs):
        tokens = df['Token'].tolist()
        token_frequencies.update(tokens)

        for token in tokens:
            if is_valid_token(token, token_frequencies):
                term_document_matrix[doc_id][token] += 1

    for doc_id, terms in term_document_matrix.items():
        term_document_matrix[doc_id] = {token: freq for token, freq in terms.items() if is_valid_token(token, token_frequencies)}

    return token_frequencies, term_document_matrix

In [30]:
token_frequencies, term_document_matrix = get_freqs(train)
token_frequencies.most_common(20)

[('the', 19776),
 ('to', 11324),
 ('of', 9649),
 ('a', 8956),
 ('and', 8355),
 ('in', 7903),
 ('on', 4855),
 ('said', 4676),
 ('s', 4645),
 ('that', 3932),
 ('for', 3710),
 ('The', 2725),
 ('is', 2580),
 ('Trump', 2537),
 ('with', 2478),
 ('was', 2264),
 ('by', 2239),
 ('he', 2185),
 ('has', 2134),
 ('as', 2093)]

In [36]:
term_document_matrix[0]

{'Reuters': 2,
 'The': 6,
 'Communications': 2,
 'expected': 2,
 '2015': 5,
 'net': 6,
 'neutrality': 7,
 'rules': 10,
 'internet': 13,
 'Pai': 10,
 'FCC': 11,
 'President': 2,
 'Internet': 3,
 'said': 10,
 'comment': 2,
 'approved': 2,
 'early': 2,
 'could': 2,
 'meeting': 2,
 'providers': 4,
 'access': 2,
 'others': 2,
 'service': 2,
 'like': 2,
 'T': 2,
 'Inc': 5,
 'N': 2,
 'O': 5,
 'make': 2,
 'group': 2,
 'open': 4,
 'regulatory': 2,
 'Congress': 2,
 'adopt': 2,
 'legislation': 2,
 'Thursday': 2,
 'framework': 2}

In [40]:
data_dir = Path('../assets/data/')
data_dir.mkdir(parents=True, exist_ok=True)
with open(data_dir / 'token_frequencies.tsv', 'w', encoding='utf-8') as file:
    for token, freq in token_frequencies.items():
        if is_valid_token(token, token_frequencies):
            file.write(f'{token}\t{freq}\n')

with open(data_dir / 'term_document_matrix.tsv', 'w', encoding='utf-8') as file:
    for doc_id, terms in term_document_matrix.items():
        for token, freq in terms.items():
            file.write(f'{doc_id}\t{token}\t{freq}\n')

In [45]:
def preprocess_text(text):
    def split_into_sentences(text):
        # so the website will not split into two separate sentences by comma:
        sentence_endings = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)(?=\s|[#])')
        sentences = sentence_endings.split(text)
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        return sentences
    
    def split_into_words(sentences):
        # regular expression to match complex URLs, simple URLs, hashtags, Twitter handles, and words
        word_pattern = re.compile(r'pic.twitter.com/\S+|https?://\S+|www\.\S+|\#\S+|\@\w+|\b\w+\'?\w*|-?\w+\'?\w*')
        tokenized_sentences = []
        for sentence in sentences:
            words = word_pattern.findall(sentence)
            tokenized_sentences.append(words)
        return tokenized_sentences

    sentences = split_into_sentences(text)
    tokenized = split_into_words(sentences)
    return tokenized

In [108]:
def compute_tf(sentence_tokens, token):
    return sentence_tokens.count(token) / len(sentence_tokens)

def compute_idf(token, term_document_matrix, total_documents):
    doc_count = sum(1 for doc in term_document_matrix if token in term_document_matrix[doc])
    return log1p(total_documents / (1 + doc_count))

def process_text_and_create_matrices(text, token_frequencies, term_document_matrix):
    tokenized_sentences = preprocess_text(text)
    total_documents = len(term_document_matrix)
    vocabulary = sorted(token_frequencies.keys())

    max_sentence_length = max(len(sentence) for sentence in tokenized_sentences)

    frequency_matrix = []
    tfidf_matrix = []

    for sentence in tokenized_sentences:
        sentence_freq_vector = [0] * max_sentence_length
        sentence_tfidf_vector = [0] * max_sentence_length

        for i, token in enumerate(sentence):
            if token in vocabulary:
                tf = compute_tf(sentence, token)
                idf = compute_idf(token, term_document_matrix, total_documents)

                sentence_freq_vector[i] = tf
                sentence_tfidf_vector[i] = tf * idf

        frequency_matrix.append(sentence_freq_vector)
        tfidf_matrix.append(sentence_tfidf_vector)

    frequency_matrix = np.array(frequency_matrix)
    tfidf_matrix = np.array(tfidf_matrix)

    document_vector_freq = np.mean(frequency_matrix, axis=0)
    document_vector_tfidf = np.mean(tfidf_matrix, axis=0)

    return document_vector_freq, document_vector_tfidf

In [109]:
text = 'Boos and chants of  Lock her up!  were heard in the crowd assembled at the West Front of the U.S. Capitol Friday morning when defeated Democratic Party presidential nominee Hillary Clinton was introduced at the inaugural ceremony for President-elect Donald Trump.#InaugurationDay Lock her up pic.twitter.com/APVtyyYote  Bill Simms (@Mittens1245) January 20, 2017The crowd on the mall booed when the jumbotron showed a close-up shot of Hillary Clinton at #Inauguration https://t.co/1dvY5lxdKo  gpbnews (@gpbnews) January 20, 2017Some in crowd chanting LOCK HER UP as Hillary Clinton arrives  Jamie Dupree (@jamiedupree) January 20, 2017Via: Gateway Pundit '
print(text)

Boos and chants of  Lock her up!  were heard in the crowd assembled at the West Front of the U.S. Capitol Friday morning when defeated Democratic Party presidential nominee Hillary Clinton was introduced at the inaugural ceremony for President-elect Donald Trump.#InaugurationDay Lock her up pic.twitter.com/APVtyyYote  Bill Simms (@Mittens1245) January 20, 2017The crowd on the mall booed when the jumbotron showed a close-up shot of Hillary Clinton at #Inauguration https://t.co/1dvY5lxdKo  gpbnews (@gpbnews) January 20, 2017Some in crowd chanting LOCK HER UP as Hillary Clinton arrives  Jamie Dupree (@jamiedupree) January 20, 2017Via: Gateway Pundit 


In [110]:
document_vector_freq, document_vector_tfidf = process_text_and_create_matrices(text, token_frequencies, term_document_matrix)

In [111]:
document_vector_freq.shape, document_vector_tfidf.shape

((53,), (53,))

In [112]:
document_vector_freq

array([0.00925926, 0.05687831, 0.06316762, 0.09094539, 0.00925926,
       0.06316762, 0.06613757, 0.03703704, 0.02812718, 0.02812718,
       0.00925926, 0.04961565, 0.01554857, 0.02183788, 0.01554857,
       0.01554857, 0.01554857, 0.02183788, 0.00925926, 0.01554857,
       0.01554857, 0.01554857, 0.01554857, 0.01554857, 0.01554857,
       0.02183788, 0.02183788, 0.02480783, 0.03703704, 0.00925926,
       0.00925926, 0.00925926, 0.02812718, 0.02812718, 0.00925926,
       0.01554857, 0.01257862, 0.00628931, 0.        , 0.        ,
       0.        , 0.00628931, 0.01257862, 0.01257862, 0.00628931,
       0.00628931, 0.        , 0.        , 0.01886792, 0.01886792,
       0.        , 0.        , 0.        ])

In [113]:
document_vector_tfidf

array([0.06396995, 0.36087457, 0.43640956, 0.62831942, 0.03438493,
       0.4006411 , 0.45692823, 0.25587981, 0.08231434, 0.0928415 ,
       0.06396995, 0.3025914 , 0.0521641 , 0.09587016, 0.083105  ,
       0.05262563, 0.07649802, 0.15087253, 0.03596311, 0.03260675,
       0.06286926, 0.03069588, 0.04526071, 0.04654449, 0.06608678,
       0.09547157, 0.06550397, 0.17139119, 0.25587981, 0.06396995,
       0.04369899, 0.06396995, 0.06049447, 0.07434251, 0.01124828,
       0.05451776, 0.0467116 , 0.03909815, 0.        , 0.        ,
       0.        , 0.04345129, 0.03150162, 0.0307501 , 0.03655433,
       0.03475128, 0.        , 0.        , 0.050955  , 0.04541578,
       0.        , 0.        , 0.        ])

In [ ]:
model = gensim.models.Word2Vec(sentences=train, vector_size=100, window=5, min_count=5, workers=4)

model.save('word2vec.model')